In [9]:
import json

def generate_jsonld(json_data, json_schema):
    # Load JSON schema
    with open(json_schema, 'r') as f_schema:
        schema = json.load(f_schema)
    
    # Prepare context from schema
    context = {
        "@context": {
            "schema": "http://schema.org/"
        }
    }
    for prop, value in schema["items"]["properties"].items():
        if prop == "text":
            for lang_prop in value["properties"]:
                context["@context"][lang_prop] = "schema:" + lang_prop
        else:
            context["@context"][prop] = "schema:" + prop

    # Prepare JSON-LD graph
    graph = {"@graph": []}
    for item in json_data:
        entity = {"@type": "schema:LocalBusiness"}
        for prop, value in item.items():
            if prop == "text":
                entity[prop] = {lang: value[lang] for lang in value}
            else:
                entity[prop] = value
        graph["@graph"].append(entity)

    # Combine context and graph
    jsonld = {**context, **graph}
    return jsonld

def main():
    json_data_file = "coopcycle.json"
    json_schema_file = "coopcycle_schema.json"
    output_jsonld_file = "coopcycle.jsonld"

    # Load JSON data
    with open(json_data_file, 'r') as f_data:
        json_data = json.load(f_data)

    # Generate JSON-LD
    jsonld = generate_jsonld(json_data, json_schema_file)

    # Write JSON-LD to file
    with open(output_jsonld_file, 'w') as f_jsonld:
        json.dump(jsonld, f_jsonld, indent=4)

if __name__ == "__main__":
    main()
